In [1]:
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
import os,pidly,glob,re
from astropy.io import fits
from astropy.wcs import WCS
from scipy.interpolate import RegularGridInterpolator

Sample from Kamenetzky et al. (2016) https://iopscience.iop.org/article/10.3847/0004-637X/829/2/93#apjaa22eas2

In [2]:
sample=[]
herschel_table=np.loadtxt('references/Kamenetzky_herschel.csv',delimiter=',',dtype=str)
basic_table=np.loadtxt('references/Kamenetzky_basic.csv',delimiter=',',dtype=str)
apex_basic=np.loadtxt('references/basic_info.csv',delimiter=',',dtype=str)
for i in range(len(herschel_table)):
    if herschel_table[i,1].strip()=='CI1-0' and herschel_table[i,6].strip()=='':
        sample.append(herschel_table[i,0].strip())
apex_cord=[]
repeat=[]
for i in range(len(apex_basic)-1):
    c2=SkyCoord(apex_basic[i+1,1],apex_basic[i+1,2],frame="icrs",unit=(u.hourangle, u.deg))
    apex_cord.append(c2)
for i in range(len(sample)):
    for j in range(len(basic_table)):
        if basic_table[j,0].strip()==sample[i]:
            tmp=basic_table[j,2:8]
            c1 = SkyCoord(':'.join(tmp[:3]),':'.join(tmp[3:]), frame="icrs",unit=(u.hourangle, u.deg))
            break
    for k in range(len(apex_cord)):
        #print(c1,apex_cord[k])
        if c1.separation(apex_cord[k])<0.1*u.deg:
            repeat.append(sample[i])
for i in repeat:
    sample.remove(i)
apex_cord_str=np.zeros([len(apex_basic)-1,2]).astype(str)
Kam_cord_str=np.zeros([len(sample),2]).astype(str)
for i in range(len(apex_basic)-1):
    apex_cord_str[i,0]=apex_basic[i+1,1]
    apex_cord_str[i,1]=apex_basic[i+1,2]
for i in range(len(sample)):
    for j in range(len(basic_table)):
        if basic_table[j,0].strip()==sample[i]:
            idx=j
            break
    Kam_cord_str[i,0]=':'.join(basic_table[idx,2:5]).replace(' ','')
    Kam_cord_str[i,1]=':'.join(basic_table[idx,5:8]).replace(' ','')
#sample=np.concatenate([apex_basic[1:,0],sample])
apex_samp=np.concatenate([[apex_basic[1:,0]],apex_cord_str.T]).T
Kam_samp=np.concatenate([[sample],Kam_cord_str.T]).T
sample_2=np.concatenate([apex_samp,Kam_samp])

In [3]:
Montoya_basic=np.loadtxt('references/Montoya_basic.csv',delimiter=',',dtype=str)
Montoya_flux=np.loadtxt('references/Montoya_flux.csv',delimiter=',',dtype=str)
Mo_name=[]
for i in range(len(Montoya_flux)):
    if len(Montoya_flux[i,7].strip())>0:
        Mo_name.append(Montoya_flux[i,0].strip())
Mon_cord_str=np.zeros([len(Mo_name),2]).astype(str)
for i in range(len(Mo_name)):
    for j in range(1,len(Montoya_basic)):
        if Montoya_basic[j,0].strip()==Mo_name[i]:
            idx=j
            break
    Mon_cord_str[i,0]=Montoya_basic[idx,2].strip()
    Mon_cord_str[i,1]=Montoya_basic[idx,3].strip()
Mon_samp=np.concatenate([[Mo_name],Mon_cord_str.T]).T
sample_3=np.concatenate([sample_2,Mon_samp])

In [4]:
coord=[]
repeat=[]
for i in range(len(sample_3)):
    ci=SkyCoord(sample_3[i,1],sample_3[i,2],frame="icrs",unit=(u.hourangle, u.deg))
    isin=False
    for c in coord:
        if ci.separation(c)<0.001*u.deg:
            isin=True
            break
    if not isin:
        coord.append(ci)
    else:
        repeat.append(i)
idx=np.delete(np.arange(len(sample_3)),repeat)
final_sample=sample_3[idx]

In [5]:
def write_flux(flux,mol_name,file_path):
    if not os.path.exists(file_path):
        tmp=np.concatenate([[[mol_name,'error','beam','ref']],flux])
        np.savetxt(file_path,tmp,fmt='%s',delimiter=',')
    else:
        ori=np.loadtxt(file_path,delimiter=',',dtype=str)
        if mol_name not in ori[0]:
            tmp=np.concatenate([[[mol_name,'error','beam','ref']],flux])
            a1,a2,b1,b2=ori.shape[0],ori.shape[1],tmp.shape[0],tmp.shape[1]
            result=np.empty([max(a1,b1),a2+b2],dtype='U64')
            result[:a1,:a2]=ori
            result[:b1,a2:a2+b2]=tmp
            np.savetxt(file_path,result,fmt='%s',delimiter=',')
#flux=np.array([[1,2,3,4],[5,6,7,8],[8,0,87,8]])
#write_flux(flux,'CI2-1','NGC1.csv')

In [6]:
idl=pidly.IDL('idl')
def beam_flux(hdu,ra,dec,beam):
    w=WCS(hdu[1].header)
    c=SkyCoord(ra,dec,frame="icrs",unit=(u.hourangle, u.deg))
    data=hdu[1].data
    pixel=w.world_to_pixel(c)
    x,y=np.arange(data.shape[0]),np.arange(data.shape[1])
    f=RegularGridInterpolator((x,y),data,method='linear')
    if hdu[1].header['BUNIT']=='MJy/sr':
        ufactor=beam**2/37550.9
    elif hdu[1].header['BUNIT']=='Jy/pixel':
        ufactor=1.133*beam**2/abs(hdu[1].header['CDELT1']*hdu[1].header['CDELT2']*3600**2)
    flux=float(f(pixel[::-1]))*ufactor
    return flux
def beam_factor(name,orgbeam,targbeam):
    idx=np.where(basic_info[:,0]==name)[0][0]
    ra,dec=basic_info[idx,1],basic_info[idx,2]
    name=name.replace(' ','')
    for beam in [orgbeam,targbeam]:
        if os.path.exists('convolve/Images/%s_160_to_%3.1f.fits'%(name,beam)):
            continue
        kerfile='Kernel_HiRes_PACS_160_to_Gauss_%3.1f'%beam
        if not os.path.exists('convolve/Kernels/'+kerfile+'.fits'):
            parfile='/DATA/zj/CI_gasmass/dust_SED/photometry/make_kernel/Parameters/Parameters_file_PACS160_to_tmp.dat'
            f=open(parfile,'r',encoding='utf-8')
            flist=f.readlines()
            flist[17]="Gauss %3.1f\n"%beam
            flist[18]='Gauss_%3.1f           Filename of the PSF\n'%beam
            flist[23]='%3.1f                 Gaussian  FWHM\n'%beam
            f=open(parfile,'w',encoding='utf-8')
            f.writelines(flist)
            f.close()
            idl.filename='Parameters_file_PACS160_to_tmp'
            idl('.compile /DATA/zj/CI_gasmass/dust_SED/photometry/make_kernel/IDL_Routines/make_kernel_v21.pro')
            idl('make_kernel_v21,filename')
            os.system('cp /DATA/zj/CI_gasmass/dust_SED/photometry/make_kernel/Kernels/Ker_PACS160_to_tmp/Kernel_HiRes_PACS_160_to_Gauss_%3.1f.fits.gz convolve/Kernels/'%beam)
            os.system('gzip -d convolve/Kernels/*.gz')
        idl.kername=kerfile
        idl.imagename='%s_160'%name
        os.system('cp convolve/Herschel_raw/sorted/%s/%s_160.fits convolve/Images/'%(name,name))
        idl('.compile convolve/IDL_Routines/convolve_image.pro')
        idl('convolve_image,kername,imagename')
        os.system('mv convolve/Images/%s_160_convolved.fits convolve/Images/%s_160_to_%3.1f.fits'%(name,name,beam))
        os.system('rm convolve/Images/%s_160_kernel.fits'%name)
    
    hdu=fits.open('convolve/Images/%s_160_to_%3.1f.fits'%(name,orgbeam))
    orgflux=beam_flux(hdu,ra,dec,orgbeam)
    hdu=fits.open('convolve/Images/%s_160_to_%3.1f.fits'%(name,targbeam))
    targflux=beam_flux(hdu,ra,dec,targbeam)
    factor=targflux/orgflux
#beam_factor('NGC1068',23,43.5)

In [7]:
print(apex_basic[0])
print(basic_table[0])
print(Montoya_basic[0])

['name' 'ra' 'dec' 'redshift' 'SIMBAD name' 'type' 'Distance' 'obsid'
 'F70-35' 'F70-30' 'F70-17' 'FWHM' 'Size / “']
['NGC0023         ' '   ' ' 0 ' '09' ' 53.36' ' +25' ' 55' ' 26.4' ' 10.9'
 '   68' '  5' '  7' ' 1342247622' ' 1342234681  ']
[' Galaxy name        ' ' z        ' ' RA            ' ' Dec            '
 ' $\\alpha_{\\textrm{AGN}}$ ' ' $\\log L_{\\textrm{IR}}$ '
 ' $\\log L_{\\textrm{AGN}}$ ' ' SFR               '
 ' OH$_{\\rm max}^{\\dagger}$ ' ' OH$_{\\rm EQW}^{\\dagger}$ ' ' Ref.']


## herschel dust continuum ##

In [9]:
'''
os.system('rm -r convolve/Herschel_raw/untared/*')
tars=glob.glob('convolve/Herschel_raw/*tar')
for tar in tars:
    os.system('tar xvf %s -C convolve/Herschel_raw/untared/'%(tar))
os.system('mv convolve/Herschel_raw/untared/*/* convolve/Herschel_raw/untared/')
'''

"\nos.system('rm -r convolve/Herschel_raw/untared/*')\ntars=glob.glob('convolve/Herschel_raw/*tar')\nfor tar in tars:\n    os.system('tar xvf %s -C convolve/Herschel_raw/untared/'%(tar))\nos.system('mv convolve/Herschel_raw/untared/*/* convolve/Herschel_raw/untared/')\n"

In [10]:
'''
raw_ids=glob.glob('convolve/Herschel_raw/untared/*')
raw_pos=[]
extnames=['HPPHPF*','HPPJS*','HPPUNI*','HPPP*','extd*W','psrc*W']
for raw_id in raw_ids:
    infofit=glob.glob(raw_id+'/*fits.gz')
    header=fits.open(infofit[0])[0].header
    raw_pos.append([header['RA'],header['DEC']])
for i in range(len(basic_info)):
    c=SkyCoord(basic_info[i,1],basic_info[i,2],frame="icrs",unit=(u.hourangle, u.deg))
    sou_pos=[c.ra.deg,c.dec.deg]
    os.system('rm -r convolve/Herschel_raw/sorted/%s'%basic_info[i,0].replace(' ',''))
    os.system('mkdir convolve/Herschel_raw/sorted/%s'%basic_info[i,0].replace(' ',''))
    for j in range(len(raw_ids)):
        dis=np.sqrt((sou_pos[0]-raw_pos[j][0])**2+(sou_pos[1]-raw_pos[j][1])**2)
        if dis<1/12:
            path1=glob.glob(raw_ids[j]+'/level2*')[-1]
            for extname in extnames:
                path2s=glob.glob(path1+'/'+extname)
                if len(path2s)!=0:
                    for path2 in path2s:
                        path3=glob.glob('%s/*.fits.gz'%(path2))[0]
                        wl=fits.open(path3)[0].header['WAVELNTH']
                        os.system('cp %s convolve/Herschel_raw/sorted/%s/%s_%d.fits.gz'%(path3,basic_info[i,0].replace(' ',''),basic_info[i,0].replace(' ',''),int(wl)))
                    break


def search_dir(path):
    path1=glob.glob(path+'/*')
    for p in path1:
        if '.' not in p.split('/')[-1]:
            path2=p
            break
    return path2
raw_ids=glob.glob('convolve/Spitzer_raw/untared/*')
raw_pos=[]
for raw_id in raw_ids:
    infofits=glob.glob(raw_id+'/ch*/')
    infofit=infofits[np.argmin([int(s[-2]) for s in infofits])]
    header=fits.open(glob.glob(infofit+'/pbcd/*maic.fits')[0])[0].header
    raw_pos.append([header['RA_REF'],header['DEC_REF']])
for i in range(len(basic_info)): 
    c=SkyCoord(basic_info[i,1],basic_info[i,2],frame="icrs",unit=(u.hourangle, u.deg))
    sou_pos=[c.ra.deg,c.dec.deg]
    dis=[]
    for j in range(len(raw_ids)):
        dis.append(np.sqrt((sou_pos[0]-raw_pos[j][0])**2+(sou_pos[1]-raw_pos[j][1])**2))
    idx=np.argmin(dis)
    if dis[idx]<1/20:
        infofits=glob.glob(raw_ids[idx]+'/ch*/')
        infofit=infofits[np.argmin([int(s[-2]) for s in infofits])]
        fitsnames=glob.glob(infofit+'pbcd/*000*maic.fits')
        fitsname=fitsnames[np.argmin([int(re.search('_00(\d)(\d)_',fitname).group(2)) for fitname in fitsnames])]
        print(basic_info[i,0],fitsname)
        hdu=fits.open(fitsname)
        wl=int(round((hdu[0].header['NOMWAVE'])))
        os.system('cp %s convolve/Herschel_raw/sorted/%s/%s_%d.fits'%(fitsname,basic_info[i,0].replace(' ',''),basic_info[i,0].replace(' ',''),int(wl)))
'''         

'\nraw_ids=glob.glob(\'convolve/Herschel_raw/untared/*\')\nraw_pos=[]\nextnames=[\'HPPHPF*\',\'HPPJS*\',\'HPPUNI*\',\'HPPP*\',\'extd*W\',\'psrc*W\']\nfor raw_id in raw_ids:\n    infofit=glob.glob(raw_id+\'/*fits.gz\')\n    header=fits.open(infofit[0])[0].header\n    raw_pos.append([header[\'RA\'],header[\'DEC\']])\nfor i in range(len(basic_info)):\n    c=SkyCoord(basic_info[i,1],basic_info[i,2],frame="icrs",unit=(u.hourangle, u.deg))\n    sou_pos=[c.ra.deg,c.dec.deg]\n    os.system(\'rm -r convolve/Herschel_raw/sorted/%s\'%basic_info[i,0].replace(\' \',\'\'))\n    os.system(\'mkdir convolve/Herschel_raw/sorted/%s\'%basic_info[i,0].replace(\' \',\'\'))\n    for j in range(len(raw_ids)):\n        dis=np.sqrt((sou_pos[0]-raw_pos[j][0])**2+(sou_pos[1]-raw_pos[j][1])**2)\n        if dis<1/12:\n            path1=glob.glob(raw_ids[j]+\'/level2*\')[-1]\n            for extname in extnames:\n                path2s=glob.glob(path1+\'/\'+extname)\n                if len(path2s)!=0:\n             

In [14]:
targbeam=43.5
idx=[]
basic_info=np.zeros([len(final_sample),5]).astype(str)
basic_info[:,:3]=final_sample
for i in range(len(basic_info)):
    paths=glob.glob('convolve/Herschel_raw/sorted/%s/*'%(basic_info[i,0].replace(' ','')))
    #for path in paths:
    #    os.system('gzip -d %s'%path)
    if len(paths)<5:
        idx.append(i)
final_basic=np.delete(basic_info,idx,0)

apex_SLED=np.loadtxt('references/CO_SLED.csv',delimiter=',',dtype=str)
#print(final_basic)
def match_str_list(tstr,olist):
    result=[]
    for i in range(len(olist)):
        if olist[i].strip()==tstr.strip():
            result.append(i)
    return result
for i in range(len(final_basic)):
    for j in range(1,len(apex_basic)):
        if apex_basic[j,0].strip()==final_basic[i,0]:
            final_basic[i,3]=apex_basic[j,3]
            final_basic[i,4]=apex_basic[j,6]
            idx=np.where(apex_SLED==final_basic[i,0])[0][0]
            flux=np.empty([13,4],dtype='U64')
            flux[:3,:]=apex_SLED[idx,1:13].reshape([3,4])
            flux[3:,0],flux[3:,1]=apex_SLED[idx,13:33:2],apex_SLED[idx,14:34:2]
            flux[3:,2]=43.5
            flux[3:,3]='Lu et al. 2017'
            nan_idx=12
            for k in range(12,0,-1):
                if flux[k,1]=='nan':
                    nan_idx=k-1
                else:
                    break
            flux=flux[:nan_idx+1]
            #os.system('rm line_luminositeis/'+final_basic[i,0]+'.csv')
            write_flux(flux,'CO','line_luminositeis/'+final_basic[i,0]+'.csv')
            flux=np.empty([2,4],dtype='U64')
            flux[0,:2]=apex_SLED[idx,37:39]
            flux[1,:2]=apex_SLED[idx,35:37]
            flux[0,2],flux[0,3]='13.5','this work'
            flux[1,2],flux[1,3]='43.5','Lu et al. 2017'
            write_flux(flux,'CI','line_luminositeis/'+final_basic[i,0]+'.csv')
            break
    if float(final_basic[i,3])!=0:
        continue
    #Kamenetzky table
    for j in range(len(basic_table)):
        if basic_table[j,0].strip()==final_basic[i,0]:
            final_basic[i,3]=np.nan
            final_basic[i,4]=basic_table[j,9]
            flux=np.empty([13,4],dtype='U64')
            lowJCO=np.loadtxt('references/Kamenetzky_lowJ.csv',delimiter=',',dtype='str')
            idx1=match_str_list(final_basic[i,0],lowJCO[:,0])
            flux=np.empty([13,4],dtype='U64')
            for Jlow in range(3):
                idx2=match_str_list(str(Jlow+1),lowJCO[idx1,1])
                idxs=np.array(idx1)[idx2]
                lineflux=[]
                for idx in idxs:
                    if lowJCO[idx,9].strip()!='':
                        lineflux.append(list(lowJCO[idx,8:10].astype(float)))
                    elif lowJCO[idx,5].strip()=='Jykms':
                        if lowJCO[idx,7].strip()!='':
                            factor=beam_factor(final_basic[i,0],float(lowJCO[idx,7]),targbeam)
                        else:
                            factor=1
                        lineflux.append([float(lowJCO[idx,2]),np.sqrt(float(lowJCO[idx,3])**2+float(lowJCO[idx,4])**2)])
                lineflux=np.array(lineflux)
                if len(lineflux)==0:
                    lflux,lerror=np.nan,np.nan
                else:
                    lflux=np.mean(lineflux[:,0])
                    lerror=np.sqrt((np.sqrt(np.sum(lineflux[:,1]**2))/len(lineflux))**2+np.std(lineflux[:,0])**2)/2
                flux[Jlow,:2]=[lflux,lerror]
            highJCO=np.loadtxt('references/Kamenetzky_herschel.csv',delimiter=',',dtype=str)
            idx1=match_str_list(final_basic[i,0],highJCO[:,0])
            for Jlow in range(3,13):
                idx2=match_str_list('CO%d-%d'%(Jlow+1,Jlow),highJCO[idx1,1])
                idx=np.array(idx1)[idx2]
                if len(idx)==0:
                    lflux,lerror=np.nan,np.nan
                elif highJCO[idx[0],6].strip()=='':
                    lflux=float(highJCO[idx[0],3])
                    lerror=(float(highJCO[idx[0],5])-float(highJCO[idx[0],4]))/2
                else:
                    lflux=np.nan
                    lerror=float(highJCO[idx[0],6])
                flux[Jlow,:2]=[lflux,lerror]
            flux[:,2]='43.5'
            flux[:,3]='Kamenetzky et al. 2016'
            write_flux(flux,'CO','line_luminositeis/'+final_basic[i,0]+'.csv')
            flux=np.empty([2,4],dtype='U64')
            idx1=match_str_list(final_basic[i,0],highJCO[:,0])
            for Jlow in range(2):
                idx2=match_str_list('CI%d-%d'%(Jlow+1,Jlow),highJCO[idx1,1])
                idx=np.array(idx1)[idx2]
                if len(idx)==0:
                    lflux,lerror=np.nan,np.nan
                elif highJCO[idx[0],6].strip()=='':
                    lflux=float(highJCO[idx[0],3])
                    lerror=(float(highJCO[idx[0],5])-float(highJCO[idx[0],4]))/2
                else:
                    lflux=np.nan
                    lerror=float(highJCO[idx[0],6])
                flux[Jlow,:2]=[lflux,lerror]
            flux[:,2]='43.5'
            flux[:,3]='Kamenetzky et al. 2016'
            write_flux(flux,'CI','line_luminositeis/'+final_basic[i,0]+'.csv')
            break
    if float(final_basic[i,3])!=0:
        continue
    for j in range(1,len(Montoya_basic)):
        if Montoya_basic[j,0].strip()==final_basic[i,0]:
            final_basic[i,3]=Montoya_basic[j,1]
            final_basic[i,4]=np.nan
            COCIflux=np.loadtxt('references/Montoya_flux.csv',delimiter=',',dtype=str)
            idx1=match_str_list(final_basic[i,0],COCIflux[:,0])
            flux=np.empty([3,4],dtype='U64')
            for Jlow in range(3):
                lfluxstr=COCIflux[idx1[0],1+2*Jlow]
                if lfluxstr.strip()!='':
                    lflux,lerror=np.array(lfluxstr.split('$\\pm$')).astype(float)
                else:
                    lflux,lerror=np.nan,np.nan
                flux[Jlow,:2]=[lflux,lerror]
            flux[:,3]='Montoya Arroyave et al.2023'
            #write_flux(flux,'CO','line_luminositeis/'+final_basic[i,0]+'.csv')
            flux=np.empty([1,4],dtype='U64')
            lfluxstr=COCIflux[idx1[0],7]
            if lfluxstr.strip()!='':
                lflux,lerror=np.array(lfluxstr.split('$\\pm$')).astype(float)
            else:
                lflux,lerror=np.nan,np.nan
            flux[0,:2]=[lflux,lerror]
            flux[0,3]='Montoya Arroyave et al.2023'
            #write_flux(flux,'CI','line_luminositeis/'+final_basic[i,0]+'.csv')
            print(final_basic[i,0])

IRAS F12112+0305
IRAS F14348-1447
IRAS 17208-0014
IRAS 19542+1110
IRAS F20551-4250
IRAS F22491-1808
IRAS F23128-5919
IRAS F00509+1225
IRAS F12243-0036


In [17]:
#np.savetxt('references/total_info.csv',np.concatenate([[['source name','RA','Dec','redshift','Distance']],final_basic]),fmt='%s',delimiter=',')

In [18]:
final_basic
infofile=np.loadtxt('references/total_info.csv',delimiter=',',dtype=str)
idx=[]
for i in range(1,len(infofile)):
    if infofile[i,0] not in final_basic[:,0]:
        idx.append(i)
afterrm=np.delete(infofile,idx,0)
#np.savetxt('references/total_info_v2.csv',afterrm,delimiter=',',fmt='%s')

array([['NGC1068', '02:42:40.55', '-00:00:46.60', '0.00379', '10.1'],
       ['IC1623AB', '01:07:47.42', '-17:30:25.00', '0.02007', '85.6'],
       ['NGC0232A', '00:42:45.82', '-23:33:41.70', '0.02264', '88.4'],
       ['ESO244', '01:18:08.26', '-44:27:42.90', '0.021905', '91.1'],
       ['NGC1614', '04:33:59.98', '-08:34:43.70', '0.01594', '70.5'],
       ['IRAS09022', '09:04:12.72', '-36:27:01.30', '0.05964', '280'],
       ['IRAS08355', '08:37:01.86', '-49:54:30.00', '0.0259', '120'],
       ['ESO339', '19:57:37.59', '-37:56:08.50', '0.0192', '66.4'],
       ['NGC6926', '20:33:06.08', '-02:01:38.60', '0.019847', '83.8'],
       ['NGC3110A', '10:04:02.09', '-06:28:28.60', '0.01686', '72.7'],
       ['IRAS13120', '13:15:06.28', '-55:09:22.90', '0.03076', '143'],
       ['UGC02238', '02:46:17.50', '13:05:44.00', '0.02188', '95'],
       ['MCG-05-12-00', '04:52:04.90', '-32:59:26.00', '0.01875', '84.1'],
       ['UGC03094', '04:35:33.90', '19:10:18.30', '0.02471', '107'],
       ['NGC15